<a href="https://colab.research.google.com/github/RoshaniPawar16/AAI/blob/main/Task1/Deep_Q_Learning_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827618 sha256=d968a3e4fceb7d9a43268e46cec2f44b0557885d79799c1ddccfcc6bd10ffa6d
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.0.9 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


In [ ]:
import gym
from gym.envs.toy_text.frozen_lake import generate_random_map
import time
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from tqdm import tqdm
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
np.random.seed(4)

# Create the environment with is_slippery=True
random_map = generate_random_map(size=10, p=0.3)
env = gym.make("FrozenLake-v1", desc=random_map, render_mode="rgb_array", is_slippery=True)
env.reset()
env.render()

# Named tuple representing a single transition in our environment
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'done'))

nspace = env.observation_space.n
naction = env.action_space.n
print(nspace)

# Cyclic buffer of bounded size that holds and samples the transitions observed recently
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.priorty = []
        self.position = 0
        self.alpha = 0.6

    def push(self, *args, step):
        """Saves a transition."""
        priority = (step + 1e-5) ** self.alpha

        if len(self.memory) < self.capacity:
            self.memory.append(None)
            self.priorty.append(priority)
        else:
            self.priorty[self.position] = priority
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity
        # self.priorty[self.position] = priority

    def sample(self, batch_size):
        # return random.sample(self.memory, batch_size)
        indices = np.random.choice(len(self.memory), batch_size, replace=False, p=self.priorty / np.sum(self.priorty))
        return [self.memory[i] for i in indices]

    def __len__(self):
        return len(self.memory)

class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()

        self.network = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.LeakyReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 256),
            nn.LeakyReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 256),
            nn.LeakyReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 256),
            nn.LeakyReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, action_dim)
            # nn.Linear(state_dim, action_dim)
        )

        # Initialize weights using Xavier/Glorot initialization
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        return self.network(x)

def get_action(state, epsilon):
    exp_exp_tradeoff = random.uniform(0, 1)
    if exp_exp_tradeoff > epsilon:
        with torch.no_grad():
            action = policy_net(state).max(1)[1].view(1, 1)
    else:
        # action = torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)
        action = env.action_space.sample()
        action = torch.tensor(np.array(action), device=device).view(1, 1)
    return action

# Hyperparameters
total_episodes = 50000*2
max_steps = 500
learning_rate = 1e-4
gamma = 0.6
batch_size = 64
target_update = 50
train_frequency = 10
train_epochs = 10
goal_position = env.observation_space.n - 1  # Goal is the last position in the grid

# Exploration parameters
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.0001/5

policy_net = DQN(nspace, naction).to(device)
target_net = DQN(nspace, naction).to(device)

target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.AdamW(policy_net.parameters(), lr=learning_rate, weight_decay=1e-5)

# Initialize the replay buffer
memory = ReplayMemory(500000)

def one_hot_encode(state, num_classes):
    one_hot_array = np.zeros(num_classes, dtype=np.float32)
    one_hot_array[state] = 1
    return one_hot_array

# Manhattan distance heuristic to guide the agent towards the goal
def manhattan_distance(state):
    grid_size = int(np.sqrt(nspace))
    state_x, state_y = state % grid_size, state // grid_size
    goal_x, goal_y = goal_position % grid_size, goal_position // grid_size
    return abs(goal_x - state_x) + abs(goal_y - state_y)


def trainDQN(policy_net, target_net, optimizer, epsilon, total_episodes, memory):
    rewards = []
    steps_moved = []
    new_rewards = []
    for episode in tqdm(range(1, total_episodes + 1)):
        current_state, _ = env.reset()
        state = one_hot_encode(current_state, nspace)
        state = torch.tensor(np.array([state]), device=device)
        total_rewards = 0
        actual_rewards = 0
        total_steps = 0
        visited_states = set()
        visited_states.add(current_state)
        for step in range(max_steps):
            action = get_action(state, epsilon)
            next_state, org_reward, done, _, _ = env.step(action.item())

            # Reward shaping (adjusted for simplicity)
            if done:
                reward = 100 if org_reward == 1 else -100
            else:
                # Reward for getting closer to the goal
                reward = 0
                distance_to_goal = manhattan_distance(current_state)
                next_distance_to_goal = manhattan_distance(next_state)

                if next_distance_to_goal < distance_to_goal:
                    reward = 2.0  # Double the reward for moving closer
                elif next_distance_to_goal > distance_to_goal:
                    reward = -0.1 * next_distance_to_goal  # Minor penalty for moving away from the goal

                # Additional checks to avoid loops and repeated states
                if next_state in visited_states or next_state < current_state:
                    reward -= 1.0  # Penalty for revisiting states to avoid loops
                else:
                    reward += 0.5  # Small reward for exploring new states

                visited_states.add(next_state)  # Track visited states to prevent loops

            next_state = one_hot_encode(next_state, nspace)
            total_rewards += org_reward
            actual_rewards += reward
            total_steps += 1


            reward = torch.tensor(np.array([reward]), device=device)
            done = torch.tensor(np.array([int(done)]), device=device)
            next_state = torch.tensor(np.array([next_state]), device=device)

            with torch.no_grad():
                current_q = policy_net(state).gather(1, action)
                next_q = target_net(next_state).max(1)[0] * (1 - done)
                td_error = abs(current_q - (reward + gamma * next_q)).item()


            memory.push(state, action, next_state, reward, done, step=td_error)


            state = next_state

            if done:
                break

        if episode % train_frequency == 0 and len(memory) >= batch_size:
            for _ in range(train_epochs):
                transitions = memory.sample(batch_size)
                batch = Transition(*zip(*transitions))

                state_batch = torch.cat(batch.state)
                action_batch = torch.cat(batch.action)
                next_state_batch = torch.cat(batch.next_state)
                reward_batch = torch.cat(batch.reward)
                done_batch = torch.cat(batch.done)

                # Current Q values for chosen actions
                state_action_values = policy_net(state_batch).gather(1, action_batch)

                # Double DQN: action selection by policy_net, evaluation by target_net
                with torch.no_grad():
                    next_actions = policy_net(next_state_batch).max(1)[1].unsqueeze(1)
                    next_state_values = target_net(next_state_batch).gather(1, next_actions).squeeze()

                # Calculate the target Q values
                target_q_values = (next_state_values * (1 - done_batch) * gamma) + reward_batch

                # Loss calculation and backpropagation
                loss = F.smooth_l1_loss(state_action_values, target_q_values.unsqueeze(1))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        if episode % target_update == 0:
            target_net.load_state_dict(policy_net.state_dict())

        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        rewards.append(total_rewards)
        steps_moved.append(total_steps)
        new_rewards.append(actual_rewards)
        if episode % 1000 == 0:
            print(f"Episode {episode}: Reward = {total_rewards}, Epsilon = {epsilon}, Steps = {total_steps}, Adjusted Reward = {actual_rewards}")
            os.makedirs("checkpoints", exist_ok=True)
            torch.save(policy_net.state_dict(), f'checkpoints/policy_net-{episode}.pth')
            torch.save(target_net.state_dict(), f'checkpoints/target_net-{episode}.pth')

    print("Score over time: " + str(sum(rewards) / total_episodes))
    fig, axes = plt.subplots(3, 2, figsize=(15, 12))
    fig.tight_layout(pad=4.0)

    # Plot Reward at Each Episode
    axes[0, 0].plot(rewards)
    axes[0, 0].set_xlabel('Episode')
    axes[0, 0].set_ylabel('Reward')
    axes[0, 0].set_title('Reward at Each Episode')

    # Plot Steps Moved at Each Episode
    axes[0, 1].plot(steps_moved)
    axes[0, 1].set_xlabel('Episode')
    axes[0, 1].set_ylabel('Steps Moved')
    axes[0, 1].set_title('Steps Moved at Each Episode')

    # Plot Adjusted Reward at Each Episode
    axes[1, 0].plot(new_rewards)
    axes[1, 0].set_xlabel('Episode')
    axes[1, 0].set_ylabel('Adjusted Reward')
    axes[1, 0].set_title('Adjusted Reward at Each Episode')

    # Plot Reward at Each Episode (Moving Average)
    moving_average = np.convolve(rewards, np.ones(1000) / 1000, mode='valid')
    axes[1, 1].plot(moving_average)
    axes[1, 1].set_xlabel('Episode')
    axes[1, 1].set_ylabel('Reward')
    axes[1, 1].set_title('Reward at Each Episode (Moving Average)')

    # Plot Adjusted Reward at Each Episode (Moving Average)
    moving_average = np.convolve(new_rewards, np.ones(1000) / 1000, mode='valid')
    axes[2, 0].plot(moving_average)
    axes[2, 0].set_xlabel('Episode')
    axes[2, 0].set_ylabel('Reward')
    axes[2, 0].set_title('Adjusted Reward at Each Episode (Moving Average)')

    # Plot Steps Moved at Each Episode (Moving Average)
    moving_average = np.convolve(steps_moved, np.ones(1000) / 1000, mode='valid')
    axes[2, 1].plot(moving_average)
    axes[2, 1].set_xlabel('Episode')
    axes[2, 1].set_ylabel('Steps Moved')
    axes[2, 1].set_title('Steps Moved at Each Episode (Moving Average)')

    plt.show()

    return rewards

# For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob
import base64, io, os

os.environ['SDL_VIDEODRIVER']='dummy'

os.makedirs("video", exist_ok=True)

def show_video(env_name):
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = 'video/{}.mp4'.format(env_name)
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

def show_video_of_model(env_name):
    vid = video_recorder.VideoRecorder(env, path="video/{}.mp4".format(env_name))
    state, prob = env.reset()
    state = one_hot_encode(state, nspace)
    state = torch.tensor(np.array([state]), device=device)
    done = False
    epsilon = 0
    tl = 0
    max_steps = 500
    policy_net.load_state_dict(torch.load(f'checkpoints/policy_net-{total_episodes}.pth'))
    for t in range(max_steps+1):
        vid.capture_frame()
        with torch.no_grad():
            action = policy_net(state).max(1)[1].view(1, 1)
        next_state, reward, done, info, _ = env.step(action.item())
        next_state = one_hot_encode(next_state, nspace)
        tl+=reward
        next_state = torch.tensor(np.array([next_state]), device=device)
        # print(f"state: {state}, action: {action.item()}",next_state, reward, done)
        state = next_state
        # break in the next iteration

    vid.close()
    env.close()
    print("Final reward:", tl)
    return tl

rewards = trainDQN(policy_net, target_net, optimizer, epsilon, total_episodes, memory)

show_video_of_model("FrozenLake-v1")

show_video("FrozenLake-v1")

100


  1%|          | 1006/100000 [00:49<1:04:44, 25.49it/s]

Episode 1000: Reward = 0.0, Epsilon = 0.9803966865736877, Steps = 4, Adjusted Reward = -96.0


  2%|▏         | 2006/100000 [01:25<1:00:01, 27.21it/s]

Episode 2000: Reward = 0.0, Epsilon = 0.9611815447608, Steps = 3, Adjusted Reward = -95.0


...

 50%|█████     | 50002/100000 [1:15:42<1:47:06,  7.78it/s]

Episode 50000: Reward = 0.0, Epsilon = 0.3742006467597279, Steps = 16, Adjusted Reward = -76.5


 51%|█████     | 51005/100000 [1:17:46<1:27:55,  9.29it/s]

Episode 51000: Reward = 0.0, Epsilon = 0.3669889907713475, Steps = 19, Adjusted Reward = -76.5


 51%|█████▏    | 51324/100000 [1:18:26<1:24:05,  9.65it/s]

In [13]:
show_video_of_model("FrozenLake-v1")

show_video("FrozenLake-v1")

<ipython-input-12-dad2e771865c>:334: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy_net.load_state_dict(torch.load(f'checkpoints/policy_net-{total_episodes}.pth'))


Moviepy - Building video video/FrozenLake-v1.mp4.
Moviepy - Writing video video/FrozenLake-v1.mp4



Moviepy - Done !
Moviepy - video ready video/FrozenLake-v1.mp4
Final reward: 0.0
